### Read y Write en Delta Lake
- Escribir datos en Delta Lake (Managed)
- Escribir datos en Delta Lake (External)
- Leer datos desde delta Lake (Table)
- Leer datos desde Del lake (File)

In [0]:
%sql
create schema if not exists movie_demo
location "/mnt/datamoviehistory/demo/"

In [0]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DoubleType, DateType

In [0]:
movie_schema = StructType(
    fields = [
        StructField("movieId", IntegerType(), False),
        StructField("title", StringType(), True),
        StructField("budget", IntegerType(), True),
        StructField("homepage", StringType(), True),
        StructField("overview", StringType(), True),
        StructField("popularity", DoubleType(), True),
        StructField("yearReleaseDate", IntegerType(), True),
        StructField("releaseDate", DateType(), True),
        StructField("revenue", DoubleType(), True),
        StructField("durationTime", IntegerType(), True),
        StructField("movieStatus", StringType(), True),
        StructField("tagline", StringType(), True),
        StructField("voteAverage", DoubleType(), True),
        StructField("voteCount", IntegerType(), True)
    ]
)

In [0]:
movie_df = spark.read.option("header", True).schema(movie_schema).csv(
    "/mnt/datamoviehistory/bronze/2024-12-30/movie.csv"
)

In [0]:
movie_df.write.format("delta").mode("overwrite").saveAsTable("movie_demo.movies_managed")

In [0]:
#movie_df.write.mode("overwrite").format("delta").save("/mnt/datamoviehistory/demo/movies_managed")

In [0]:
movie_df.write.mode("overwrite").format("delta").save("/mnt/datamoviehistory/demo/movies_external")

In [0]:
%sql
create table if not exists movie_demo.movies_external
using delta
location "/mnt/moviehistory/demo/movies_external"

In [0]:
%sql
--select * from movie_demo.movies_external;


In [0]:
movies_external_df = spark.read.format("delta").load("/mnt/datamoviehistory/demo/movies_external")

In [0]:
movie_df.write.format("delta").mode("overwrite").partitionBy("yearReleaseDate").saveAsTable("movie_demo.movies_partitioned")

In [0]:
%sql
-- show partitions movie_demo.movies_partitioned

### Update and Delete en Delta Lake

- Update desde Delta Lake
- Delete desde Delta Lake

In [0]:
%sql
--select * from movie_demo.movies_managed

In [0]:
%sql
UPDATE movie_demo.movies_managed SET durationTime = 60 WHERE yearReleaseDate = 2012;

In [0]:
%sql
--select * from movie_demo.movies_managed where yearReleaseDate = 2012

In [0]:
from delta.tables import * 

deltaTable = DeltaTable.forPath(spark, '/mnt/datamoviehistory/demo/movies_managed')
deltaTable.update(
    condition = "yearReleaseDate = 2013",
    set = {
        "durationTime": "100"
    }
)


In [0]:
%sql

delete from movie_demo.movie_managed
where yearReleaseDate = 2014;

In [0]:
from delta.tables import * 

deltaTable = DeltaTable.forPath(spark, '/mnt/datamoviehistory/demo/movies_managed')
deltaTable.delete(
    condition = "yearReleaseDate = 2015"
)

### Merge/Upsert en Delta Lake